In [216]:
import nltk
import json

In [217]:
content = []
for line in open('JACS.json', 'r'):
    content.append(json.loads(line))

In [218]:
Total = []
for c in content:
    ##using both title and content
    #total = c['Title'] + " " + c['Content']
    
    ##using only the title
    total = c['Title']
    Total.append(total.replace('.'," ").replace(','," ").replace(';'," ").replace(":"," ").replace("-"," "))

In [220]:
import collections # optional, but we found the collections.Counter object useful
import numpy as np
import scipy.sparse as sp
from nltk.corpus import stopwords


def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
    
    Returns: tuple: (tfidf, all_words)
        tfidf: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for 
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """
    all_words = []
    document_words = [doc.split() for doc in docs]
    
    vocab = sorted(set(sum([doc.split() for doc in docs], [])))
    vocab_dict = {k:i for i,k in enumerate(vocab)}
    count_number = {k:0 for _,k in enumerate(vocab)}
    count_word = {k:0 for _,k in enumerate(vocab)}
    row = []
    column = []
    d = []
    w = []

    for i,doc in enumerate(document_words):
        for key, value in count_number.items():
            if key in doc:
                count_number[key] += 1
     
        
    for key, value in count_number.items():
        count_number[key] = np.log(len(document_words)/count_number[key])

    
    for i,doc in enumerate(document_words):
        for word in doc:
            count_word[word] += 1
        
        for key,value in count_word.items():
            if value > 0:
                d.append(count_number[key]*value)
                w.append(key)
                row.append(i)
                column.append(vocab_dict[key])
            if key not in all_words:
                all_words.append(key)
        count_word = {k:0 for _,k in enumerate(vocab)}
    
    wordlist = []
    stopword = stopwords.words('english')
    for word in w:
        if word not in stopword:
             wordlist.append(word)
    cx = sp.coo_matrix((d,(row,column)),shape = (len(document_words),len(vocab_dict)))
    cx.eliminate_zeros()
    return (d,sorted(w))
            

In [221]:
X_tfidf, words = tfidf(Total)
print(len(X_tfidf))
print(len(words))

14588
14588


In [222]:
##Get rid of common stopwords 
from nltk.corpus import stopwords
d = {}
stopword = stopwords.words('english')
for i, t in enumerate(words):
    if t.lower() not in stopword and t.lower().isalpha():
        d[t] = X_tfidf[i]

In [223]:
##Keep only the nouns
final = {}
for key in d:
# function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'
# do the nlp stuff
    tokenized = nltk.word_tokenize(key)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
    if len(nouns) == 1:
        final[nouns[0]] = d[nouns[0]]

In [224]:
##Sort the dictionary 
import operator
sorted_d = sorted(final.items(), key=operator.itemgetter(1))

In [225]:
sorted_d

[('Acetylene', 0.5292183107510516),
 ('Acid', 0.5292183107510516),
 ('Activation', 0.5292183107510516),
 ('Affinity', 0.5292183107510516),
 ('Aldol', 0.5292183107510516),
 ('Allostery', 0.5292183107510516),
 ('Ammonium', 0.5292183107510516),
 ('Amyloid', 0.5292183107510516),
 ('Antibody', 0.5292183107510516),
 ('Azabuckybowl', 0.5292183107510516),
 ('Bacillus', 0.5292183107510516),
 ('Basis', 0.5292183107510516),
 ('Benzannulation', 0.5292183107510516),
 ('Biomarker', 0.5292183107510516),
 ('Bonds', 0.5292183107510516),
 ('Breast', 0.5292183107510516),
 ('Bulk', 0.5292183107510516),
 ('Capture', 0.5292183107510516),
 ('Cavity', 0.5292183107510516),
 ('Chirality', 0.5292183107510516),
 ('Converse', 0.5292183107510516),
 ('CuI', 0.5292183107510516),
 ('Delivers', 0.5292183107510516),
 ('Deuteration', 0.5292183107510516),
 ('Diagram', 0.5292183107510516),
 ('Diols', 0.5292183107510516),
 ('Disorder', 0.5292183107510516),
 ('Distances', 0.5292183107510516),
 ('Electric', 0.5292183107510516